In [ ]:
!pip install textblob
!pip install textstat
!pip install nltk
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install sklearn

!pip install afinn

^C
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/spacy/__main__.py", line 31, in <module>
    plac.call(commands[command], sys.argv[1:])
  File "/anaconda3/lib/python3.6/site-packages/plac_core.py", line 328, in call
    cmd, result = parser.consume(arglist)
  File "/anaconda3/lib/python3.6/site-packages/plac_core.py", line 207, in consume
    return cmd, self.func(*(args + varargs + extraopts), **kwargs)
  File "/anaconda3/lib/python3.6/site-packages/spacy/cli/download.py", line 36, in download
    .format(m=model_name, v=version), pip_args)
  File "/anaconda3/lib/python3.6/site-packages/spacy/cli/download.py", line 86, in download_model
    return subprocess.call(cmd, env=os.environ.copy())
  File "/anaconda3/lib/python3.6/subprocess.py", line 269, in cal

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
import pandas as pd
from textblob import TextBlob
import textstat
import nltk
import spacy
from collections import Counter
import en_core_web_sm
import os
import sklearn
import afinn

In [179]:
df = pd.read_csv(r'C:\Users\C18244A\Documents\FlipFlops_Propaganda_Project\datatask2_df_test.csv',
                     usecols=['sentences', 'article', 'N_sentence', 'is_propaganda', 
                              'sentiment_polarity', 'sentiment_subjectivity'])

In [180]:
df.columns

Index(['sentences', 'article', 'N_sentence', 'is_propaganda',
       'sentiment_polarity', 'sentiment_subjectivity'],
      dtype='object')

In [181]:
df.shape

(4042, 6)

In [182]:
df.head()

,sentences,article,N_sentence,is_propaganda,sentiment_polarity,sentiment_subjectivity
0,Next plague outbreak in Madagascar could be 's...,111111111,1,?,0.000000,0.000000
1,Geneva - The World Health Organisation chief o...,111111111,3,?,-0.066667,0.466667
2,"""The next transmission could be more pronounce...",111111111,5,?,0.055556,0.388889
3,"An outbreak of both bubonic plague, which is s...",111111111,7,?,0.150000,0.250000
4,Madagascar has suffered bubonic plague outbrea...,111111111,9,?,0.000000,0.000000


In [183]:
def apply_nlp(row):
    nlp = en_core_web_sm.load()
    temp = defaultdict(int)
    temp.update(Counter(map(lambda x:x.label_, nlp(row).ents)))
    return temp

def iter_row(series):
    return [apply_nlp(s) for s in series]

def generate_features(df):
    df['sentiment_polarity'] = df.apply(lambda x: TextBlob(x['sentences']).sentiment[0], axis=1)
    df['sentiment_subjectivity'] = df.apply(lambda x: TextBlob(x['sentences']).sentiment[1], axis=1)

    data = df.apply(lambda x: apply_nlp(x['sentences']), axis=1)
    df_POS = pd.DataFrame(data.tolist())

    df = df.merge(df_POS, left_index=True, right_index=True)
    
    df['flesch_reading_ease'] = df.apply(lambda x: textstat.flesch_reading_ease(x['sentences']), axis=1)
    df['smog_index'] = df.apply(lambda x: textstat.smog_index(x['sentences']), axis=1)
    df['flesch_kincaid_grade'] = df.apply(lambda x: textstat.flesch_kincaid_grade(x['sentences']), axis=1)
    df['coleman_liau_index'] = df.apply(lambda x: textstat.coleman_liau_index(x['sentences']), axis=1)
    df['automated_readability_index'] = df.apply(lambda x: textstat.automated_readability_index(x['sentences']), axis=1)
    df['dale_chall_readability_score'] = df.apply(lambda x: textstat.dale_chall_readability_score(x['sentences']), axis=1)
    df['difficult_words'] = df.apply(lambda x: textstat.difficult_words(x['sentences']), axis=1)
    df['linsear_write_formula'] = df.apply(lambda x: textstat.linsear_write_formula(x['sentences']), axis=1)
    df['gunning_fog'] = df.apply(lambda x: textstat.gunning_fog(x['sentences']), axis=1)
    df['text_standard'] = df.apply(lambda x: textstat.text_standard(x['sentences']), axis=1)

    return df

In [ ]:
df = generate_features(df)

In [ ]:
import pandas as pd
import re
from itertools import chain
import sys
from nltk import tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer


class DataTokenization:

    def __init__(self, tickets):
        """
        Init DataTokenization object

        Args:
            ds: Dataframe (pandas type) having only index and one column with single document per record.
        """

        if not isinstance(tickets, list):
            raise Exception('The input should be of type list')

        self.tickets = tickets
        self.use_stemming = "Yes"
        self.n_grams_type = 'simple'
        self.n_grams = "3"
        self.stopwords = stopwords.words('english')
        self.use_phrases = "Yes"

    def _document_tokenizer(self, document: str, m_stopwords: list, wordnet_lemmatizer,
                            stemmer, synonyms_method):
        """
        Args:
            document: a document. Could be from single word to multiple lines/paragraphs text.
            m_stopwords [Optional]: This is a list of the stopwords which should be excluded from the tokens
                                  The default value is empty list which means no words will be excluded.
            stemmer: [object] Stemmming object.         
            wordnet_lemmatizer: [object] Lemmatizer object.         
            synonyms_method [Optional]: How the synonyms are exctacted - using nltk or spacy

        Returns:
            A list of prepared tokens of the documnet.

        Example:
            input document:
            'John.Doe@examplemail.com cannot log onto the SDE (or reset the password).
            Emma raised ART request  for an SDE login for John.Doe@examplemail.com
            This request has status "COMPLETE", but in the "COMMENTS" field there is a message "Failed to add user Example.name to PORT_T_NEW"

            outputted tokens:
            ['log', 'onto', 'sde', 'reset', 'password', 'rais', 'request', 'sde', 'login',
            'request', 'status', 'complet', 'comment', 'messag', 'fail', 'add', 'user', 'examplename', 'porttnew']
        """     
        
        try:
            # split string into words (tokens)
            tokens = tokenize.word_tokenize(document)   

            # remove any digits, i.e. "3rd edition"; remove any path, i.e. "//math/lib" and remove any path on PC
            tokens = [t for t in tokens if not any((c.isdigit() or c == '/' or c == '\\') for c in t)] 

            # remove special chars
            tokens = [re.sub(r'[^a-zA-Z0-9\s]', '', t) for t in tokens]  
            
            # remove short words which don't contain capital letters. 
            tokens = [t for t in tokens 
                      if not (len(t) <= 2 and t != 'no' and t != 'nt' and (t.islower()) or t == '')] 
            
            # downcase
            tokens = [t.lower() for t in tokens]  
            
            # remove stopwords
            tokens = [t for t in tokens if t not in m_stopwords]  
            
            # put words into base form
            tokens = [wordnet_lemmatizer.lemmatize(t, 'v') for t in tokens]  

            # put words into core form
            # execute only if no synonyms are needed
            # if both stemming and synonyms should be added - this is added in visual script
            stremming = self.use_stemming.lower().strip()
            phrases = elf.use_phrases.lower().strip()
            if ((stremming == 'yes' or stremming == 'y') and (phrases != 'yes' or phrases!= 'y')):

                tokens = [stemmer.stem(t) for t in tokens]  

        except Exception as err:
            raise Exception("There was an issue with tokenizing a document:" + str(err))

        return tokens

    def _phrase_generator(self, wordnet_lemmatizer, stemmer):
        """
        This method generates the phrases based on prepared tickets from 
        _document_tokenizer method.

        Args:
            wordnet_lemmatizer: [object] Lemmatizer object. 

         Returns:
            A list of prepared phreses of the corpus.        
        """

        corpus_all_phrases = \
            [self._document_tokenizer(self.tickets[doc], m_stopwords=self.stopwords,
             wordnet_lemmatizer=wordnet_lemmatizer, stemmer='', synonyms_method='none') 
             for doc in range(len(self.tickets))]

        docs = [' '.join([t for t in doc]) for doc in corpus_all_phrases]

        # set spaCy object with wordnet annotator
        nlp = spacy.load('en_core_web_sm')

        bad_deps = ('prep', 'aux')
        phrase_list = []
        phrase_list_per_doc = []

        for doc in docs:
            # take the phases using spaCy object with noun_chunks attribute
            phrase_list_per_doc = []
            for phrase in nlp(doc).noun_chunks:
               
                # exclude bad_deps
                while len(phrase) > 1 and phrase[0].dep_ not in bad_deps:
                    phrase = phrase[1:]

                # check if our phrase have one or more word and            
                if len(phrase) > 1 and len(str(phrase))>1:
                    # Merge the tokens, e.g. good_ideas
                    phrase = str(phrase.text).lower().replace(" ", "_")
                    phrase_list_per_doc.append(phrase)
        
                if  len(phrase_list_per_doc) > 0:    
                    phrase_list.append(phrase_list_per_doc)
                else:
                     phrase_list.append("None")
        return phrase_list

    def tickets_tokenizer(self):
        # load object from the related packages - Lemmatizer
        wordnet_lemmatizer = WordNetLemmatizer()

        # load object from the related packages - Stemming
        if (self.use_stemming.lower().strip() == 'yes' or self.use_stemming.lower().strip() == 'y'):
            stemmer = SnowballStemmer("english")
        else:
            stemmer = ''

        # Perform the tokenization
        corpus_all_tokens = \
            [self._document_tokenizer(self.tickets[doc], m_stopwords=self.stopwords,
             wordnet_lemmatizer=wordnet_lemmatizer, stemmer=stemmer, synonyms_method='') 
             for doc in range(len(self.tickets))]

        corpus_all_tokens = [' '.join(doc) for doc in corpus_all_tokens]
        corpus_all_tokens = [doc if str(doc).strip() != '' else 'test' for doc in corpus_all_tokens]

        # Create a list of list per token for bi-grams
        n_grams_2, n_grams_3 = [], []

        if (int(self.n_grams) != 0 and int(self.n_grams) != 2 and int(self.n_grams) != 3):
            raise Exception(("You need to enter valid value for n-grams: 0, 2 or 3 as "
                            "integer value.s \n You have used {self.n_grams} !"))        

        if (int(self.n_grams) == 2 or int(self.n_grams) == 3):

            if (self.n_grams_type.lower().strip() == 'simple'):
                n_grams_2 = [[a + "_" + b for a,b in ngrams(tokens.split(), 2)] for tokens in corpus_all_tokens]

                if (int(self.n_grams) == 3): 
                    n_grams_3 = [[a + "_" + b + "_" + c for a,b,c in ngrams(tokens.split(), 3)] for tokens in corpus_all_tokens]
            
        n_grams_2 = [' '.join(doc) for doc in n_grams_2]      
        n_grams_3 = [' '.join(doc) for doc in n_grams_3]

        # Phrases 
        if (self.use_stemming.lower().strip() == 'yes'
            or self.use_stemming.lower().strip() == 'y'):

            try:
                phrase_list = list(self._phrase_generator(wordnet_lemmatizer = wordnet_lemmatizer, stemmer = stemmer))
                phrase_list = [' '.join(set_phrases) if set_phrases != "None" else set_phrases for set_phrases in phrase_list]
            except Exception as err:
                raise Exception("There was an issue with separatig the key phreses:" + str(err))

        return (corpus_all_tokens, n_grams_2, n_grams_3, phrase_list)

In [ ]:
# Load the tickets data
data = df

# take the text unstructured part of the dataset
# this df format with the index from the original dataset
documents = data['sentences'].tolist()

# load the DataPreprocessing object - the main DataParser object (by now)
data_parser_object = DataTokenization(documents)

corpus_all_tokens, n_grams_2, n_grams_3, phrase_list = data_parser_object.tickets_tokenizer()

corpus_all_tokens_df = pd.DataFrame(corpus_all_tokens, columns=['corpus_all_tokens'])
corpus_all_tokens_df.to_csv("corpus_all_tokens_df2.csv")

n_grams_2_df = pd.DataFrame(n_grams_2, columns=['n_grams_2'])
n_grams_2_df.to_csv("n_grams_2_df2.csv")

n_grams_3_df = pd.DataFrame(n_grams_3, columns=['n_grams_3'])
n_grams_3_df.to_csv("n_grams_3_df2.csv")

phrase_list_df= pd.DataFrame(phrase_list, columns=['phrase_list'])
phrase_list_df.to_csv("phrase_list2.csv")


In [143]:
! ls

'Combine Semantic Features with Emotional Features.ipynb'
 corpus_all_tokens_df2.csv
'Dataprep for task 2-3.ipynb'
 dataprepT23.py
'Download the data.ipynb'
'Feature Creations.ipynb'
 final_data.pkl
"Fix Ogi's problem.ipynb"
 Merge_Preped_Data_Case2.ipynb
 n_grams_2_df2.csv
 n_grams_3_df2.csv
 phrase_list2.csv
 __pycache__
'R Demo.ipynb'
 Untitled.ipynb
'Update dataprep for Dobi.ipynb'


In [144]:
combined_dataset = pd.concat([corpus_all_tokens_df, n_grams_2_df, n_grams_3_df], axis=1, join = 'inner')
combined_dataset.to_csv("corpus_all_tokens_n_grams_df2.csv")

## Calculate afinn score

In [ ]:
af = afinn.Afinn()

df['affin_on_article'] = df.apply(lambda x: af.score(text=x['article']), axis=1)

In [151]:
# combined_dataset = pd.read_csv('../generated_data/Ogi\'s folder/new data/corpus_all_tokens_n_grams_df2.csv', 
#                                             usecols=['corpus_all_tokens', 'n_grams_2', 'n_grams_3'])

corpus_all_tokens_n_grams_df2 = combined_dataset
corpus_all_tokens_n_grams_df2.shape
corpus_all_tokens_n_grams_df2.head()

### Load semantic variables

#semantic_vars_df = pd.read_csv("../generated_data/Ogi\'s folder/new data/df_task2_with_features.csv")
semantic_vars_df = df
semantic_vars_df.shape
semantic_vars_df.head(5)
semantic_vars_df.columns

Index(['sentences', 'article', 'N_sentence', 'is_propaganda', 'affin_score',
       'sentiment_polarity', 'sentiment_subjectivity'],
      dtype='object')

In [152]:
#drop_columns = ['Unnamed: 0', 'index', 'Unnamed: 0.1', 'Unnamed: 0.2', '0']

#semantic_vars_df = semantic_vars_df.drop(drop_columns, axis=1)

### Create total_df by joinning Corpus_Tokens and Phrase_List

In [153]:
total_df = semantic_vars_df.join(corpus_all_tokens_n_grams_df2)

total_df.head(3)

total_df.columns

total_df.shape

(14263, 10)

### Add new feature - sentece_length

In [154]:
total_df['corpus_len'] = total_df.apply(lambda x: len(x['corpus_all_tokens'].strip()), axis=1)

total_df.columns

Index(['sentences', 'article', 'N_sentence', 'is_propaganda', 'affin_score',
       'sentiment_polarity', 'sentiment_subjectivity', 'corpus_all_tokens',
       'n_grams_2', 'n_grams_3', 'corpus_len'],
      dtype='object')

In [155]:
def full_text(row):
    return ' '.join(map(str, [row['corpus_all_tokens'], row['n_grams_2'], row['n_grams_3']]))

total_df['all_tokens'] = total_df.apply(lambda x: full_text(x), axis=1)

total_df['all_tokens'][:4]

0    us bloggers ban enter uk us_bloggers bloggers_...
1    two prominent us bloggers ban enter uk home of...
2    pamela geller robert spencer cofounded antimus...
3    due speak english defence league march woolwic...
Name: all_tokens, dtype: object

### Create dictionary

In [156]:
corpus_all_tokens_unique = set([w for doc in total_df['all_tokens'] for w in doc.split()])

print("Total number of uniqure words in the corpus: " + str(len(corpus_all_tokens_unique)))

Total number of uniqure words in the corpus: 232860


### Create word2index and tf_idf dataframe

In [157]:
# encode the words in the dictionary in order to create keys DB table 
word2idx = {word:index for index, word in enumerate(corpus_all_tokens_unique)}

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_object = TfidfVectorizer(decode_error='ignore', vocabulary=word2idx)

X_tfidf = tfidf_object.fit_transform(total_df['all_tokens'])

X_tfidf

<14263x232860 sparse matrix of type '<class 'numpy.float64'>'
	with 424441 stored elements in Compressed Sparse Row format>

In [158]:
X_tfidf_df = pd.DataFrame(X_tfidf.toarray())

X_tfidf_df.shape

X_tfidf_df.head(4)

,0,1,2,3,4,5,6,7,8,9,...,232850,232851,232852,232853,232854,232855,232856,232857,232858,232859
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### ADD Afinn Score

In [159]:
data_with_afinn = pd.read_csv('../generated_data/data_case2_with_affin.csv')

data_with_afinn.head(4)

total_df.columns

Index(['sentences', 'article', 'N_sentence', 'is_propaganda', 'affin_score',
       'sentiment_polarity', 'sentiment_subjectivity', 'corpus_all_tokens',
       'n_grams_2', 'n_grams_3', 'corpus_len', 'all_tokens'],
      dtype='object')

In [160]:
data_with_afinn = data_with_afinn[['article', 'N_sentence',  'is_propaganda', 'affin_score']]

data_with_afinn.head(5)

total_df.shape

(14263, 12)

In [161]:
data_with_afinn.shape

(14263, 4)

In [162]:
new_data = total_df.merge(data_with_afinn, how='inner', on=['article', 'N_sentence',  'is_propaganda'])

new_data.shape

new_data.columns

Index(['sentences', 'article', 'N_sentence', 'is_propaganda', 'affin_score_x',
       'sentiment_polarity', 'sentiment_subjectivity', 'corpus_all_tokens',
       'n_grams_2', 'n_grams_3', 'corpus_len', 'all_tokens', 'affin_score_y'],
      dtype='object')

### Use Linear optimization

In [163]:
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components=600, n_iter=2, random_state = 1234)

X_matrix = lsa.fit_transform(X_tfidf_df) # list of list 

type(X_matrix)

X_matrix.shape

KeyboardInterrupt: 

In [ ]:
total_data = new_data.merge(pd.DataFrame(X_matrix), left_index=True, right_index=True)

total_data.shape

total_data.columns

columns = total_data.columns

print (columns[:40])

In [ ]:
# total_data = total_data.drop([ 'corpus_all_tokens', 'n_grams_2', 'n_grams_3', 'all_tokens', 'text_standard'], axis=1)

In [ ]:
total_data = total_data.fillna(0)

### Save final data for Ogi

In [ ]:
total_data.to_csv('../final_data.csv', index=False)

with open('../final_data.pkl', 'wb') as f:
    pickle.dump(total_data, f)

    '''
X = total_data.iloc[:, 4:].values

y = total_data['is_propaganda'].map({'non-propaganda':0, 'propaganda':1})

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Fitting Kernel SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train_res, y_train_res)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)

from sklearn.metrics import f1_score

f1_s = f1_score(y_test, y_pred)
print("Kernal SVM F1 result: " + str(f1_s))

import pickle

with open('../model.pkl', 'wb') as f:
    pickle.dump(classifier, f)
